In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
import glove
from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D

from keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
import np_utils
from tensorflow.python.keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

2023-11-12 23:55:53.907636: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 23:55:53.907672: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 23:55:53.907712: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 23:55:53.914029: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Reset Keras Session
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def make_prediction_timeseries(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_timeseries(predictions, probs, ground_truth, model_name, 
                problem_type, iteration, hidden_unit_size, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    
    result_dict = {} 
    result_dict['iteration'] = [iteration]
    result_dict['auc'] = [auc]
    result_dict['auprc'] = [auprc]
    result_dict['acc'] = [acc]
    result_dict['F1'] = [F1]
        
    filename = str(hidden_unit_size)+"-"+model_name+"-"+problem_type+"-"+type_of_ner+".csv"
    
    result_path = "../results/timeseries-baseline/metrics/"+filename
    df = pd.DataFrame.from_dict(result_dict)
    df.to_csv(result_path,mode='a',index=False,header=False)
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

        
    file_name = str(hidden_unit_size)+"-"+model_name+"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+".p"
    
    result_path = "../results/timeseries-baseline/metrics/"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))
    print(auc, auprc, acc, F1)

In [3]:
def timeseries_model(layer_name, number_of_unit):
    K.clear_session()
    
    sequence_input = Input(shape=(24,104),  name = "timeseries_input")
    
    if layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)
    else:
        x = GRU(number_of_unit)(sequence_input)
    
    logits_regularizer = regularizers.l2(l2=0.01)
    sigmoid_pred = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.GlorotNormal(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    model = Model(inputs=sequence_input, outputs=sigmoid_pred)
    
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [4]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("../data/timeseries/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("../data/timeseries/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("../data/timeseries/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("../data/timeseries/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("../data/timeseries/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("../data/timeseries/"+type_of_ner+"_y_test.pkl")

In [5]:
epoch_num = 100
model_patience = 3
monitor_criteria = 'val_loss'
batch_size = 128

unit_sizes = [128, 256]
#unit_sizes = [256]
iter_num = 11
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
layers = ["LSTM", "GRU"]
#layers = ["GRU"]
for each_layer in layers:
    print("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print("Hidden unit: ", each_unit_size)
        for iteration in range(1, iter_num):
            print("Iteration number: ", iteration)
            print("=============================")

            for each_problem in target_problems:
                print ("Problem type: ", each_problem)
                print ("__________________")


                early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                best_model_name = "../results/timeseries-baseline/best_models/"+str(each_layer)+"-"+str(each_unit_size)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                    save_best_only=True, mode='min', save_freq='epoch')


                callbacks = [early_stopping_monitor, checkpoint]

                model = timeseries_model(each_layer, each_unit_size)
                model.fit(x_train_lstm, y_train[each_problem], epochs=epoch_num, verbose=1, 
                          validation_data=(x_dev_lstm, y_dev[each_problem]), callbacks=callbacks, batch_size= batch_size)

                model.load_weights(best_model_name)

                probs, predictions = make_prediction_timeseries(model, x_test_lstm)
                save_scores_timeseries(predictions, probs, y_test[each_problem].values,str(each_layer),
                                       each_problem, iteration, each_unit_size,type_of_ner)
                reset_keras(model)
                #del model
                clear_session()
                gc.collect()

Layer:  LSTM
Hidden unit:  128
Iteration number:  1
Problem type:  mort_hosp
__________________


2023-11-12 23:56:04.787939: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13074 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:2b:00.0, compute capability: 7.5


Epoch 1/100


2023-11-12 23:56:06.322167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


 39/121 [========>.....................] - ETA: 0s - loss: 0.3843 - acc: 0.8512 

...

2023-11-13 00:12:25.446852: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13074 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:2b:00.0, compute capability: 7.5
2023-11-13 00:12:25.457316: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13074 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:2b:00.0, compute capability: 7.5


Problem type:  los_7
__________________
Epoch 1/100
119/121 [============================>.] - ETA: 0s - loss: 0.3172 - acc: 0.9001
Epoch 1: val_loss improved from inf to 0.26897, saving model to ../results/timeseries-baseline/GRU-256-los_7-best_model.hdf5
121/121 [==============================] - 1s 6ms/step - loss: 0.3169 - acc: 0.9000 - val_loss: 0.2690 - val_acc: 0.9205
Epoch 2/100
 55/121 [============>.................] - ETA: 0s - loss: 0.2601 - acc: 0.9210

/data/apps/miniforge3/envs/cse6250-project/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


118/121 [============================>.] - ETA: 0s - loss: 0.2606 - acc: 0.9211
Epoch 2: val_loss improved from 0.26897 to 0.26021, saving model to ../results/timeseries-baseline/GRU-256-los_7-best_model.hdf5
121/121 [==============================] - 0s 4ms/step - loss: 0.2596 - acc: 0.9215 - val_loss: 0.2602 - val_acc: 0.9219
Epoch 3/100
118/121 [============================>.] - ETA: 0s - loss: 0.2485 - acc: 0.9224
Epoch 3: val_loss improved from 0.26021 to 0.25873, saving model to ../results/timeseries-baseline/GRU-256-los_7-best_model.hdf5
121/121 [==============================] - 0s 4ms/step - loss: 0.2478 - acc: 0.9225 - val_loss: 0.2587 - val_acc: 0.9196
Epoch 4/100
104/121 [========================>.....] - ETA: 0s - loss: 0.2392 - acc: 0.9226
Epoch 4: val_loss did not improve from 0.25873
121/121 [==============================] - 0s 3ms/step - loss: 0.2404 - acc: 0.9225 - val_loss: 0.2644 - val_acc: 0.9219
Epoch 5/100
116/121 [===========================>..] - ETA: 0s - los

2023-11-13 00:12:33.383207: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13074 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:2b:00.0, compute capability: 7.5
2023-11-13 00:12:33.393354: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13074 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:2b:00.0, compute capability: 7.5
